# 🛒 E-Commerce Sales Intelligence - Complete Application

## All-in-One: Database + Analytics + Interactive Dashboard

This comprehensive notebook includes:
1. **Database Schema Creation** - 4-table normalized design
2. **Batch Data Generation** - 16,500+ realistic records
3. **Advanced SQL Queries** - Aggregations, Joins, Window Functions
4. **Interactive Dashboard** - Real-time filtering & visualizations

---

### Quick Start
**Simply run all cells** (Runtime → Run all) to create the database and launch the dashboard!

---

In [1]:
# ========================================
# SECTION 1: IMPORTS AND CONFIGURATION
# ========================================

import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
from faker import Faker
import warnings
warnings.filterwarnings('ignore')

# Visualization libraries
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Interactive widgets
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display, HTML, clear_output

# Configure pandas display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.precision', 2)

# Initialize Faker for realistic data generation
fake = Faker()
Faker.seed(42)  # For reproducibility
random.seed(42)
np.random.seed(42)

# Set plotly theme
import plotly.io as pio
pio.templates.default = "plotly_white"

print("✓ All libraries imported successfully")
print("✓ Random seeds set for reproducibility")
print("✓ Configuration complete")

✓ All libraries imported successfully
✓ Random seeds set for reproducibility
✓ Configuration complete


In [2]:
# ========================================
# SECTION 2: DATABASE CONNECTION
# ========================================

def create_database_connection(db_name='ecommerce.db'):
    """
    Establish connection to SQLite database.

    Args:
        db_name (str): Name of the database file

    Returns:
        sqlite3.Connection: Database connection object
    """
    try:
        conn = sqlite3.connect(db_name)
        print(f"✓ Connected to database: {db_name}")
        return conn
    except sqlite3.Error as e:
        print(f"✗ Database connection error: {e}")
        return None

# Create database connection
conn = create_database_connection()
cursor = conn.cursor()

✓ Connected to database: ecommerce.db


In [3]:
# ========================================
# SECTION 3: DATABASE SCHEMA CREATION (DDL)
# ========================================

def create_database_schema(conn):
    """
    Create all database tables with proper relationships and constraints.
    Implements a normalized relational schema for e-commerce operations.

    Args:
        conn (sqlite3.Connection): Active database connection
    """
    cursor = conn.cursor()

    # Drop existing tables if they exist (for clean slate)
    cursor.execute("DROP TABLE IF EXISTS reviews")
    cursor.execute("DROP TABLE IF EXISTS orders")
    cursor.execute("DROP TABLE IF EXISTS products")
    cursor.execute("DROP TABLE IF EXISTS customers")

    # TABLE 1: CUSTOMERS
    # Stores customer information including regional data for geographic analysis
    cursor.execute("""
        CREATE TABLE customers (
            customer_id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            email TEXT UNIQUE NOT NULL,
            region TEXT NOT NULL,
            registration_date DATE NOT NULL,
            customer_segment TEXT NOT NULL
        )
    """)

    # TABLE 2: PRODUCTS
    # Catalog of all products with pricing and inventory information
    cursor.execute("""
        CREATE TABLE products (
            product_id INTEGER PRIMARY KEY AUTOINCREMENT,
            product_name TEXT NOT NULL,
            category TEXT NOT NULL,
            price REAL NOT NULL CHECK(price > 0),
            stock_quantity INTEGER NOT NULL CHECK(stock_quantity >= 0),
            supplier TEXT NOT NULL
        )
    """)

    # TABLE 3: ORDERS
    # Transaction records linking customers to products with temporal data
    cursor.execute("""
        CREATE TABLE orders (
            order_id INTEGER PRIMARY KEY AUTOINCREMENT,
            customer_id INTEGER NOT NULL,
            product_id INTEGER NOT NULL,
            order_date DATE NOT NULL,
            quantity INTEGER NOT NULL CHECK(quantity > 0),
            total_amount REAL NOT NULL CHECK(total_amount > 0),
            status TEXT NOT NULL,
            FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
            FOREIGN KEY (product_id) REFERENCES products(product_id)
        )
    """)

    # TABLE 4: REVIEWS
    # Customer feedback and ratings for products
    cursor.execute("""
        CREATE TABLE reviews (
            review_id INTEGER PRIMARY KEY AUTOINCREMENT,
            product_id INTEGER NOT NULL,
            customer_id INTEGER NOT NULL,
            rating INTEGER NOT NULL CHECK(rating BETWEEN 1 AND 5),
            review_text TEXT,
            review_date DATE NOT NULL,
            FOREIGN KEY (product_id) REFERENCES products(product_id),
            FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
        )
    """)

    # Create indexes for performance optimization on frequently queried columns
    cursor.execute("CREATE INDEX idx_orders_customer ON orders(customer_id)")
    cursor.execute("CREATE INDEX idx_orders_product ON orders(product_id)")
    cursor.execute("CREATE INDEX idx_orders_date ON orders(order_date)")
    cursor.execute("CREATE INDEX idx_reviews_product ON reviews(product_id)")
    cursor.execute("CREATE INDEX idx_customers_region ON customers(region)")

    conn.commit()
    print("✓ Database schema created successfully")
    print("✓ Tables created: customers, products, orders, reviews")
    print("✓ Indexes created for query optimization")

# Execute schema creation
create_database_schema(conn)

✓ Database schema created successfully
✓ Tables created: customers, products, orders, reviews
✓ Indexes created for query optimization


In [4]:
# ========================================
# SECTION 4: BATCH DATA GENERATION
# ========================================

def generate_customers(conn, num_customers=1000):
    """
    Generate realistic customer data in batch.

    Args:
        conn: Database connection
        num_customers: Number of customers to generate
    """
    cursor = conn.cursor()

    # Define business constants
    regions = ['North', 'South', 'East', 'West', 'Central']
    segments = ['Premium', 'Standard', 'Basic']

    customers_data = []

    print(f"Generating {num_customers} customers...")

    for i in range(num_customers):
        # Generate registration date within the last 2 years
        days_ago = random.randint(0, 730)
        reg_date = (datetime.now() - timedelta(days=days_ago)).date()

        customer = (
            fake.name(),
            fake.unique.email(),
            random.choice(regions),
            reg_date,
            random.choice(segments)
        )
        customers_data.append(customer)

    # Batch insert for performance
    cursor.executemany("""
        INSERT INTO customers (name, email, region, registration_date, customer_segment)
        VALUES (?, ?, ?, ?, ?)
    """, customers_data)

    conn.commit()
    print(f"✓ {num_customers} customers inserted successfully")


def generate_products(conn, num_products=500):
    """
    Generate product catalog with realistic pricing and categories.

    Args:
        conn: Database connection
        num_products: Number of products to generate
    """
    cursor = conn.cursor()

    # Product categories and their typical price ranges
    categories = {
        'Electronics': (50, 2000),
        'Clothing': (15, 200),
        'Books': (5, 50),
        'Home': (20, 500),
        'Sports': (10, 300)
    }

    products_data = []

    print(f"Generating {num_products} products...")

    for i in range(num_products):
        category = random.choice(list(categories.keys()))
        min_price, max_price = categories[category]

        product = (
            f"{category} - {fake.catch_phrase()}",
            category,
            round(random.uniform(min_price, max_price), 2),
            random.randint(0, 1000),
            fake.company()
        )
        products_data.append(product)

    # Batch insert
    cursor.executemany("""
        INSERT INTO products (product_name, category, price, stock_quantity, supplier)
        VALUES (?, ?, ?, ?, ?)
    """, products_data)

    conn.commit()
    print(f"✓ {num_products} products inserted successfully")


def generate_orders(conn, num_orders=10000):
    """
    Generate order transactions with realistic patterns.
    Implements business logic for order status and timing.

    Args:
        conn: Database connection
        num_orders: Number of orders to generate
    """
    cursor = conn.cursor()

    # Get valid customer and product IDs
    cursor.execute("SELECT customer_id FROM customers")
    customer_ids = [row[0] for row in cursor.fetchall()]

    cursor.execute("SELECT product_id, price FROM products")
    products = {row[0]: row[1] for row in cursor.fetchall()}
    product_ids = list(products.keys())

    statuses = ['Completed', 'Pending', 'Cancelled', 'Shipped']
    status_weights = [0.7, 0.1, 0.05, 0.15]  # Most orders are completed

    orders_data = []

    print(f"Generating {num_orders} orders...")

    for i in range(num_orders):
        # Generate order date within last 365 days
        days_ago = random.randint(0, 365)
        order_date = (datetime.now() - timedelta(days=days_ago)).date()

        product_id = random.choice(product_ids)
        quantity = random.randint(1, 5)
        total_amount = round(products[product_id] * quantity, 2)

        order = (
            random.choice(customer_ids),
            product_id,
            order_date,
            quantity,
            total_amount,
            random.choices(statuses, weights=status_weights)[0]
        )
        orders_data.append(order)

    # Batch insert
    cursor.executemany("""
        INSERT INTO orders (customer_id, product_id, order_date, quantity, total_amount, status)
        VALUES (?, ?, ?, ?, ?, ?)
    """, orders_data)

    conn.commit()
    print(f"✓ {num_orders} orders inserted successfully")


def generate_reviews(conn, num_reviews=5000):
    """
    Generate product reviews with realistic ratings distribution.

    Args:
        conn: Database connection
        num_reviews: Number of reviews to generate
    """
    cursor = conn.cursor()

    # Get valid IDs
    cursor.execute("SELECT customer_id FROM customers")
    customer_ids = [row[0] for row in cursor.fetchall()]

    cursor.execute("SELECT product_id FROM products")
    product_ids = [row[0] for row in cursor.fetchall()]

    # Rating distribution (skewed towards positive)
    ratings = [1, 2, 3, 4, 5]
    rating_weights = [0.05, 0.10, 0.15, 0.35, 0.35]

    reviews_data = []

    print(f"Generating {num_reviews} reviews...")

    for i in range(num_reviews):
        days_ago = random.randint(0, 365)
        review_date = (datetime.now() - timedelta(days=days_ago)).date()

        rating = random.choices(ratings, weights=rating_weights)[0]

        # Generate review text based on rating
        if rating >= 4:
            review_text = fake.sentence(nb_words=10) + " Great product!"
        elif rating == 3:
            review_text = fake.sentence(nb_words=10) + " It's okay."
        else:
            review_text = fake.sentence(nb_words=10) + " Disappointed."

        review = (
            random.choice(product_ids),
            random.choice(customer_ids),
            rating,
            review_text,
            review_date
        )
        reviews_data.append(review)

    # Batch insert
    cursor.executemany("""
        INSERT INTO reviews (product_id, customer_id, rating, review_text, review_date)
        VALUES (?, ?, ?, ?, ?)
    """, reviews_data)

    conn.commit()
    print(f"✓ {num_reviews} reviews inserted successfully")


# Execute batch data generation
print("\n" + "="*50)
print("STARTING BATCH DATA GENERATION")
print("="*50 + "\n")

generate_customers(conn, 1000)
generate_products(conn, 500)
generate_orders(conn, 10000)
generate_reviews(conn, 5000)

print("\n" + "="*50)
print("✓ ALL DATA GENERATED SUCCESSFULLY")
print("="*50)


STARTING BATCH DATA GENERATION

Generating 1000 customers...
✓ 1000 customers inserted successfully
Generating 500 products...
✓ 500 products inserted successfully
Generating 10000 orders...
✓ 10000 orders inserted successfully
Generating 5000 reviews...
✓ 5000 reviews inserted successfully

✓ ALL DATA GENERATED SUCCESSFULLY


---

# Part 2: Interactive Dashboard

Now that the database is created and populated, let's build an interactive dashboard!

---

In [5]:
# ========================================
# SECTION 2: DATABASE CONNECTION
# ========================================

def get_database_connection():
    """
    Establish connection to the SQLite database.
    The database should be created from the analysis notebook first.

    Returns:
        sqlite3.Connection: Active database connection
    """
    # Path to the database created in the analysis folder
    db_path = 'ecommerce_complete.db'

    try:
        conn = sqlite3.connect(db_path)
        print(f"✓ Connected to database: {db_path}")
        return conn
    except sqlite3.Error as e:
        print(f"✗ Database connection error: {e}")
        print("⚠️  Please run the database_manipulation.ipynb notebook in the analysis folder first to generate the database.")
        return None

# Establish connection
# Using existing connection from above

if True:  # Connection already established in Part 1
    # Test the connection by getting table info
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    print(f"✓ Found {len(tables)} tables: {', '.join([t[0] for t in tables])}")

✓ Found 5 tables: sqlite_sequence, customers, products, orders, reviews


In [6]:
# ========================================
# SECTION 3: DATA LOADING FUNCTIONS
# ========================================

def get_filter_options(conn):
    """
    Get available filter options from the database.
    Used to populate dropdown widgets.

    Returns:
        dict: Dictionary containing lists of regions, categories, and date range
    """
    # Get unique regions
    regions = pd.read_sql_query("SELECT DISTINCT region FROM customers ORDER BY region", conn)['region'].tolist()

    # Get unique categories
    categories = pd.read_sql_query("SELECT DISTINCT category FROM products ORDER BY category", conn)['category'].tolist()

    # Get date range
    date_query = "SELECT MIN(order_date) as min_date, MAX(order_date) as max_date FROM orders"
    date_range = pd.read_sql_query(date_query, conn)

    return {
        'regions': ['All'] + regions,
        'categories': ['All'] + categories,
        'min_date': date_range['min_date'].iloc[0],
        'max_date': date_range['max_date'].iloc[0]
    }

def get_filtered_data(conn, region='All', category='All', start_date=None, end_date=None):
    """
    Retrieve data based on user-selected filters.

    Args:
        conn: Database connection
        region: Selected region (or 'All')
        category: Selected product category (or 'All')
        start_date: Start date for filtering
        end_date: End date for filtering

    Returns:
        dict: Dictionary containing filtered datasets
    """
    # Build WHERE clause based on filters
    conditions = ["o.status = 'Completed'"]

    if region != 'All':
        conditions.append(f"c.region = '{region}'")

    if category != 'All':
        conditions.append(f"p.category = '{category}'")

    if start_date:
        conditions.append(f"o.order_date >= '{start_date}'")

    if end_date:
        conditions.append(f"o.order_date <= '{end_date}'")

    where_clause = " AND ".join(conditions)

    # Query 1: Revenue by region and category (for heatmap)
    heatmap_query = f"""
        SELECT
            c.region,
            p.category,
            SUM(o.total_amount) as total_revenue,
            COUNT(o.order_id) as order_count
        FROM orders o
        INNER JOIN customers c ON o.customer_id = c.customer_id
        INNER JOIN products p ON o.product_id = p.product_id
        WHERE {where_clause}
        GROUP BY c.region, p.category
    """

    # Query 2: Daily revenue trends
    trends_query = f"""
        WITH daily_revenue AS (
            SELECT
                o.order_date,
                SUM(o.total_amount) as daily_total,
                COUNT(o.order_id) as daily_orders
            FROM orders o
            INNER JOIN customers c ON o.customer_id = c.customer_id
            INNER JOIN products p ON o.product_id = p.product_id
            WHERE {where_clause}
            GROUP BY o.order_date
        )
        SELECT
            order_date,
            daily_total,
            daily_orders,
            AVG(daily_total) OVER (
                ORDER BY order_date
                ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
            ) as moving_avg_7day
        FROM daily_revenue
        ORDER BY order_date
    """

    # Query 3: Customer Lifetime Value
    clv_query = f"""
        SELECT
            c.customer_id,
            c.name,
            c.region,
            c.customer_segment,
            COUNT(o.order_id) as total_orders,
            SUM(o.total_amount) as customer_lifetime_value,
            AVG(o.total_amount) as average_order_value
        FROM customers c
        INNER JOIN orders o ON c.customer_id = o.customer_id
        INNER JOIN products p ON o.product_id = p.product_id
        WHERE {where_clause}
        GROUP BY c.customer_id, c.name, c.region, c.customer_segment
        ORDER BY customer_lifetime_value DESC
        LIMIT 30
    """

    # Query 4: Product performance
    product_query = f"""
        SELECT
            p.product_id,
            p.product_name,
            p.category,
            p.price,
            SUM(o.quantity) as total_units_sold,
            SUM(o.total_amount) as total_revenue,
            COUNT(DISTINCT o.order_id) as total_orders,
            COALESCE(AVG(r.rating), 0) as avg_rating,
            COUNT(DISTINCT r.review_id) as review_count
        FROM products p
        INNER JOIN orders o ON p.product_id = o.product_id
        INNER JOIN customers c ON o.customer_id = c.customer_id
        LEFT JOIN reviews r ON p.product_id = r.product_id
        WHERE {where_clause}
        GROUP BY p.product_id, p.product_name, p.category, p.price
        ORDER BY total_revenue DESC
        LIMIT 20
    """

    # Query 5: Top categories by region (Window Function)
    top_category_query = f"""
        WITH regional_sales AS (
            SELECT
                c.region,
                p.category,
                SUM(o.total_amount) as total_revenue,
                COUNT(o.order_id) as total_orders,
                RANK() OVER (
                    PARTITION BY c.region
                    ORDER BY SUM(o.total_amount) DESC
                ) as category_rank
            FROM orders o
            INNER JOIN customers c ON o.customer_id = c.customer_id
            INNER JOIN products p ON o.product_id = p.product_id
            WHERE {where_clause}
            GROUP BY c.region, p.category
        )
        SELECT region, category as top_category, total_revenue, total_orders
        FROM regional_sales
        WHERE category_rank = 1
        ORDER BY total_revenue DESC
    """

    # Execute all queries
    return {
        'heatmap': pd.read_sql_query(heatmap_query, conn),
        'trends': pd.read_sql_query(trends_query, conn),
        'clv': pd.read_sql_query(clv_query, conn),
        'products': pd.read_sql_query(product_query, conn),
        'top_categories': pd.read_sql_query(top_category_query, conn)
    }


In [7]:
# ========================================
# SECTION 4: VISUALIZATION FUNCTIONS
# ========================================

def create_kpi_cards(data):
    """
    Create KPI summary cards showing key metrics.

    Args:
        data: Dictionary containing filtered datasets
    """
    # Calculate KPIs
    total_revenue = data['heatmap']['total_revenue'].sum()
    total_orders = data['heatmap']['order_count'].sum()
    avg_order_value = total_revenue / total_orders if total_orders > 0 else 0
    total_customers = len(data['clv'])
    avg_clv = data['clv']['customer_lifetime_value'].mean() if len(data['clv']) > 0 else 0

    # Create HTML for KPI cards
    kpi_html = f"""
    <style>
        .kpi-container {{
            display: flex;
            justify-content: space-around;
            margin: 20px 0;
            gap: 15px;
        }}
        .kpi-card {{
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            border-radius: 10px;
            padding: 20px;
            flex: 1;
            color: white;
            box-shadow: 0 4px 6px rgba(0,0,0,0.1);
        }}
        .kpi-card:nth-child(2) {{
            background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%);
        }}
        .kpi-card:nth-child(3) {{
            background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%);
        }}
        .kpi-card:nth-child(4) {{
            background: linear-gradient(135deg, #43e97b 0%, #38f9d7 100%);
        }}
        .kpi-title {{
            font-size: 14px;
            opacity: 0.9;
            margin-bottom: 10px;
        }}
        .kpi-value {{
            font-size: 32px;
            font-weight: bold;
        }}
    </style>
    <div class="kpi-container">
        <div class="kpi-card">
            <div class="kpi-title">Total Revenue</div>
            <div class="kpi-value">${total_revenue:,.0f}</div>
        </div>
        <div class="kpi-card">
            <div class="kpi-title">Total Orders</div>
            <div class="kpi-value">{total_orders:,}</div>
        </div>
        <div class="kpi-card">
            <div class="kpi-title">Avg Order Value</div>
            <div class="kpi-value">${avg_order_value:,.2f}</div>
        </div>
        <div class="kpi-card">
            <div class="kpi-title">Avg CLV</div>
            <div class="kpi-value">${avg_clv:,.2f}</div>
        </div>
    </div>
    """

    display(HTML(kpi_html))

def create_regional_heatmap(data):
    """
    Create interactive heatmap of sales by region and category.
    """
    df = data['heatmap']

    if len(df) == 0:
        print("⚠️  No data available for the selected filters")
        return

    pivot_data = df.pivot_table(
        index='region',
        columns='category',
        values='total_revenue',
        fill_value=0
    )

    fig = go.Figure(data=go.Heatmap(
        z=pivot_data.values,
        x=pivot_data.columns,
        y=pivot_data.index,
        colorscale='YlOrRd',
        text=np.round(pivot_data.values, 0),
        texttemplate='$%{text:,.0f}',
        textfont={"size": 10},
        colorbar=dict(title="Revenue ($)")
    ))

    fig.update_layout(
        title='📊 Sales Density Heatmap: Revenue by Region × Category',
        xaxis_title='Product Category',
        yaxis_title='Region',
        height=400,
        font=dict(size=11)
    )

    display(fig)

def create_revenue_trends(data):
    """
    Create line chart showing daily revenue trends with moving average.
    """
    df = data['trends']

    if len(df) == 0:
        print("⚠️  No data available for the selected filters")
        return

    fig = go.Figure()

    # Daily revenue
    fig.add_trace(go.Scatter(
        x=df['order_date'],
        y=df['daily_total'],
        mode='lines',
        name='Daily Revenue',
        line=dict(color='rgba(100, 149, 237, 0.4)', width=1),
        fill='tozeroy',
        fillcolor='rgba(100, 149, 237, 0.1)'
    ))

    # 7-day moving average
    fig.add_trace(go.Scatter(
        x=df['order_date'],
        y=df['moving_avg_7day'],
        mode='lines',
        name='7-Day Moving Avg',
        line=dict(color='rgb(220, 20, 60)', width=3)
    ))

    fig.update_layout(
        title='📈 Daily Revenue Trends with 7-Day Moving Average',
        xaxis_title='Date',
        yaxis_title='Revenue ($)',
        height=400,
        hovermode='x unified',
        legend=dict(x=0.01, y=0.99, bgcolor='rgba(255,255,255,0.8)')
    )

    display(fig)

def create_clv_chart(data):
    """
    Create horizontal bar chart of top customers by CLV.
    """
    df = data['clv'].head(15)

    if len(df) == 0:
        print("⚠️  No data available for the selected filters")
        return

    fig = go.Figure(data=[
        go.Bar(
            x=df['customer_lifetime_value'],
            y=df['name'],
            orientation='h',
            marker=dict(
                color=df['customer_lifetime_value'],
                colorscale='Viridis',
                showscale=True,
                colorbar=dict(title="CLV ($)", len=0.5)
            ),
            text=df['customer_lifetime_value'].round(2),
            texttemplate='$%{text:,.2f}',
            textposition='auto',
        )
    ])

    fig.update_layout(
        title='👥 Top 15 Customers by Lifetime Value (CLV)',
        xaxis_title='Customer Lifetime Value ($)',
        yaxis_title='',
        height=500,
        yaxis=dict(autorange="reversed")
    )

    display(fig)

def create_category_pie(data):
    """
    Create pie chart showing revenue distribution by category.
    """
    df = data['heatmap'].groupby('category')['total_revenue'].sum().reset_index()
    df = df.sort_values('total_revenue', ascending=False)

    if len(df) == 0:
        print("⚠️  No data available for the selected filters")
        return

    fig = go.Figure(data=[
        go.Pie(
            labels=df['category'],
            values=df['total_revenue'],
            hole=0.4,
            textinfo='label+percent',
            textposition='auto',
            marker=dict(
                colors=px.colors.qualitative.Set3,
                line=dict(color='white', width=2)
            )
        )
    ])

    fig.update_layout(
        title='🎯 Revenue Distribution by Product Category',
        height=450,
        showlegend=True,
        legend=dict(orientation="v", yanchor="middle", y=0.5, xanchor="left", x=1.1)
    )

    display(fig)

def create_product_scatter(data):
    """
    Create scatter plot of product performance: revenue vs rating.
    """
    df = data['products']

    if len(df) == 0:
        print("⚠️  No data available for the selected filters")
        return

    fig = px.scatter(
        df,
        x='avg_rating',
        y='total_revenue',
        size='total_units_sold',
        color='category',
        hover_data=['product_name', 'review_count', 'total_orders'],
        title='⭐ Product Performance: Revenue vs Average Rating',
        labels={
            'avg_rating': 'Average Customer Rating',
            'total_revenue': 'Total Revenue ($)',
            'total_units_sold': 'Units Sold'
        },
        color_discrete_sequence=px.colors.qualitative.Bold
    )

    fig.update_layout(
        height=500,
        xaxis=dict(range=[0, 5.5]),
        showlegend=True
    )

    display(fig)

def create_top_categories_table(data):
    """
    Display table of top-selling category per region (Window Function result).
    """
    df = data['top_categories']

    if len(df) == 0:
        print("⚠️  No data available for the selected filters")
        return

    # Format the dataframe for display
    df_display = df.copy()
    df_display['total_revenue'] = df_display['total_revenue'].apply(lambda x: f"${x:,.2f}")
    df_display['total_orders'] = df_display['total_orders'].apply(lambda x: f"{x:,}")

    # Create styled HTML table
    table_html = f"""
    <style>
        .results-table {{
            width: 100%;
            border-collapse: collapse;
            margin: 20px 0;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
        }}
        .results-table th {{
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 12px;
            text-align: left;
        }}
        .results-table td {{
            padding: 10px 12px;
            border-bottom: 1px solid #ddd;
        }}
        .results-table tr:hover {{
            background-color: #f5f5f5;
        }}
    </style>
    <h3>🏆 Top-Selling Category per Region (Window Function Analysis)</h3>
    {df_display.to_html(classes='results-table', index=False)}
    """

    display(HTML(table_html))

print("✓ Visualization functions loaded")

✓ Visualization functions loaded


In [8]:
# ========================================
# SECTION 6: INITIALIZE DASHBOARD FILTERS
# ========================================

# Verify connection and load filter options
if conn:
    print("✓ Using database connection from Part 1")
    print("✓ Loading filter options...")

    # Get filter options
    filter_options = get_filter_options(conn)

    print(f"\n✓ Filter options loaded:")
    print(f"  • Regions: {len(filter_options['regions'])-1}")
    print(f"  • Categories: {len(filter_options['categories'])-1}")
    print(f"  • Date range: {filter_options['min_date']} to {filter_options['max_date']}")
    print("\n✓ Dashboard ready to launch!")
else:
    print("✗ No database connection found")
    print("⚠️  Please run all cells from the beginning")
    filter_options = None

✓ Using database connection from Part 1
✓ Loading filter options...

✓ Filter options loaded:
  • Regions: 5
  • Categories: 5
  • Date range: 2024-12-05 to 2025-12-05

✓ Dashboard ready to launch!


In [ ]:
# ========================================
# SECTION 5: INTERACTIVE DASHBOARD
# ========================================

def create_dashboard(region, category, date_range):
    """
    Main dashboard function that updates all visualizations based on filters.

    Args:
        region: Selected region filter
        category: Selected category filter
        date_range: Tuple of (start_date, end_date)
    """
    start_date, end_date = date_range if date_range else (None, None)

    # Display header
    display(HTML("""
    <style>
        .dashboard-header {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 30px;
            border-radius: 10px;
            margin-bottom: 20px;
            box-shadow: 0 4px 6px rgba(0,0,0,0.1);
        }
        .dashboard-title {
            font-size: 32px;
            font-weight: bold;
            margin-bottom: 10px;
        }
        .dashboard-subtitle {
            font-size: 16px;
            opacity: 0.9;
        }
    </style>
    <div class="dashboard-header">
        <div class="dashboard-title">🛒 E-Commerce Sales Intelligence Dashboard</div>
        <div class="dashboard-subtitle">Real-time analytics with advanced SQL querying and interactive filtering</div>
    </div>
    """))

    # Display current filters
    display(HTML(f"""
    <div style="background: #f8f9fa; padding: 15px; border-radius: 5px; margin-bottom: 20px;">
        <strong>Active Filters:</strong>
        Region: <span style="color: #667eea; font-weight: bold;">{region}</span> |
        Category: <span style="color: #667eea; font-weight: bold;">{category}</span> |
        Date Range: <span style="color: #667eea; font-weight: bold;">{start_date or 'All'} to {end_date or 'All'}</span>
    </div>
    """))

    # Fetch filtered data
    data = get_filtered_data(conn, region, category, start_date, end_date)

    # Display KPI cards
    create_kpi_cards(data)

    # Display visualizations
    create_regional_heatmap(data)
    create_category_pie(data)
    create_revenue_trends(data)
    create_clv_chart(data)
    create_product_scatter(data)
    create_top_categories_table(data)

if conn and filter_options:
    # Create filter widgets
    region_dropdown = widgets.Dropdown(
        options=filter_options['regions'],
        value='All',
        description='Region:',
        style={'description_width': '100px'}
    )

    category_dropdown = widgets.Dropdown(
        options=filter_options['categories'],
        value='All',
        description='Category:',
        style={'description_width': '100px'}
    )

    # Create date options for the slider
    date_options = pd.date_range(
        start=filter_options['min_date'],
        end=filter_options['max_date'],
        freq='W'
    ).strftime('%Y-%m-%d').tolist()

    date_picker = widgets.SelectionRangeSlider(
        options=date_options,
        index=(0, len(date_options)-1),
        description='Date Range:',
        continuous_update=False,
        style={'description_width': '100px'},
        layout=widgets.Layout(width='600px')
    )

    # Create update button
    update_button = widgets.Button(
        description='🔄 Update Dashboard',
        button_style='primary',
        tooltip='Click to refresh dashboard with selected filters',
        layout=widgets.Layout(width='200px', height='40px')
    )

    # Button click handler
    def on_update_click(b):
        create_dashboard(
            region_dropdown.value,
            category_dropdown.value,
            date_picker.value
        )

    update_button.on_click(on_update_click)

    # Display controls
    display(widgets.HTML("""
        <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                    color: white; padding: 20px; border-radius: 5px; margin-bottom: 20px;">
            <h2 style="margin: 0 0 10px 0;">🎛️ Dashboard Controls</h2>
            <p style="margin: 0; opacity: 0.9;">
                Select your filters and click "Update Dashboard" to refresh the data visualization.
            </p>
        </div>
    """))

    display(region_dropdown)
    display(category_dropdown)
    display(date_picker)
    display(update_button)

    # Initial dashboard load with default filters
    create_dashboard('All', 'All', date_picker.value)
else:
    print("✗ Database connection failed. Please run all cells from the beginning.")

HTML(value='\n        <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); \n           …

Dropdown(description='Region:', options=('All', 'Central', 'East', 'North', 'South', 'West'), style=Descriptio…

Dropdown(description='Category:', options=('All', 'Books', 'Clothing', 'Electronics', 'Home', 'Sports'), style…

SelectionRangeSlider(continuous_update=False, description='Date Range:', index=(0, 51), layout=Layout(width='6…

Button(button_style='primary', description='🔄 Update Dashboard', layout=Layout(height='40px', width='200px'), …

region,top_category,total_revenue,total_orders
West,Electronics,"$819,216.04",285
Central,Electronics,"$788,742.94",284
North,Electronics,"$744,841.70",262
East,Electronics,"$615,728.15",236
South,Electronics,"$608,196.31",229


---

## Dashboard Features

### 🎯 Key Performance Indicators (KPIs)
- **Total Revenue**: Sum of all completed orders
- **Total Orders**: Count of completed transactions
- **Average Order Value**: Mean transaction value
- **Average CLV**: Mean customer lifetime value

### 📊 Visualizations

1. **Regional Sales Heatmap**: Shows revenue distribution across regions and product categories
2. **Revenue Distribution**: Pie chart showing category contribution to total revenue
3. **Daily Revenue Trends**: Line chart with 7-day moving average using window functions
4. **Customer Lifetime Value**: Top 15 customers ranked by total spending
5. **Product Performance**: Scatter plot correlating ratings with revenue
6. **Top Categories by Region**: Window function analysis using RANK()

### 🔧 Advanced SQL Features

- **Window Functions**: `RANK()`, `AVG() OVER()` for moving averages
- **Complex JOINs**: Multiple table joins (customers ⟕ orders ⟕ products ⟕ reviews)
- **Aggregations**: `SUM()`, `COUNT()`, `AVG()`, `MIN()`, `MAX()`
- **CTEs**: Common Table Expressions for query optimization
- **Subqueries**: Nested SELECT statements
- **PARTITION BY**: Regional grouping for window functions

---

## Technical Notes

**Database Location**: `../analysis/ecommerce.db`

**Dependencies**:
- SQLite3 (built-in)
- pandas
- plotly
- ipywidgets

**Performance**: All queries are optimized with indexes on frequently queried columns.

---

*Created for Database Management Course - Inha University*